In [1]:
import pandas as pd

In [2]:
Restaurent_Data = pd.read_csv("C:\\Users\\graykar\\Downloads\\Pyhton\\Data Files\\ResturentSentiment_Analysis.tsv", sep='\t')

In [3]:
pd.set_option('max_colwidth', 200)

In [4]:
Restaurent_Data.head(10)

,Review,Liked
0,I love it.,1
1,I don't Love it...,0
2,I do not Love it...,0
3,Wow... Loved this place.,1
4,Crust is not good.,0
5,Not tasty and the texture was just nasty.,0
6,Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.,1
7,The selection on the menu was great and so were the prices.,1
8,Now I am getting angry and I want my damn pho.,0
9,Honeslty it didn't taste THAT fresh.),0


In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( Restaurent_Data.iloc[: , :] , Restaurent_Data.iloc[:,1] , test_size=0.25, random_state=1)

In [7]:
X_train.shape

(752, 2)

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\graykar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
X_train['Review'].reset_index(drop=True, inplace=True)

In [51]:
X_train.head()

,Review,Liked
552,My boyfriend and I came here for the first time on a recent trip to Vegas and could not have been more pleased with the quality of food and service.,1
160,I ordered the Voodoo pasta and it was the first time I'd had really excellent pasta since going gluten free several years ago.,1
708,I work in the hospitality industry in Paradise Valley and have refrained from recommending Cibo any longer.,0
538,An excellent new restaurant by an experienced Frenchman.,1
999,Appetite instantly gone.,0


In [10]:
converted_data = []
LMT = WordNetLemmatizer()

for i in range(X_train.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', X_train['Review'][i] )
    review = review.lower()
    review = review.split() 
    review = [LMT.lemmatize(x) for x in review if not x in set(stopwords.words('english')) - {'not'}]
    review = " ".join(review)
    converted_data.append(review)

In [142]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf = TfidfVectorizer(max_features=500)

In [143]:
X = Tfidf.fit_transform(converted_data).toarray()

In [144]:
len(Tfidf.vocabulary_)

500

In [145]:
from sklearn.naive_bayes import MultinomialNB
MB = MultinomialNB()
MB.fit(X , y_train)
y_train_pred = MB.predict(X)

In [146]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import f1_score

In [147]:
cm = confusion_matrix(y_train,y_train_pred)
cm

array([[321,  47],
       [ 32, 352]], dtype=int64)

In [148]:
f1_score(y_train,y_train_pred)

0.8991060025542784

In [149]:
import pickle

In [150]:
filename = 'C:\\Users\\graykar\\Downloads\\Pyhton\\Pickel_File\\Reasturent_Sentiment_Tfidf.pkl'
pickle.dump(Tfidf, open(filename, 'wb'))

filename = 'C:\\Users\\graykar\\Downloads\\Pyhton\\Pickel_File\\Reasturent_Sentiment_Multinominal_Model.pkl'
pickle.dump(MB, open(filename, 'wb'))

In [151]:
X_test['Review'].reset_index(drop=True, inplace=True)

In [152]:
converted_test_data = []

for i in range(X_test.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', X_test['Review'][i] ).lower().split() 
    review = [LMT.lemmatize(x) for x in review if not x in set(stopwords.words('english')) - {'not'}]
    review = " ".join(review)
    converted_test_data.append(review)

In [153]:
filename = 'C:\\Users\\graykar\\Downloads\\Pyhton\\Pickel_File\\Reasturent_Sentiment_Tfidf.pkl'
tfidf_test = pickle.load(open(filename,'rb'))

In [154]:
XT = tfidf_test.transform(converted_test_data).toarray()

In [155]:
filename = 'C:\\Users\\graykar\\Downloads\\Pyhton\\Pickel_File\\Reasturent_Sentiment_Multinominal_Model.pkl'
MB_Test = pickle.load(open(filename,'rb'))

In [156]:
y_test_pred = MB_Test.predict(XT)

In [157]:
confusion_matrix(y_test , y_test_pred)

array([[96, 38],
       [25, 92]], dtype=int64)

In [158]:
f1_score(y_test , y_test_pred)

0.7449392712550607

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [33]:
log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(random_state=42)

hard_voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='hard')
hard_voting_clf.fit(X, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [35]:
from sklearn.metrics import f1_score

for clf in (log_clf, rnd_clf, svm_clf):
    clf.fit(X, y_train)
    y_pred = clf.predict(XT)
    print(clf.__class__.__name__, f1_score (y_test, y_pred))
            
hvc_predict = hard_voting_clf.predict(XT)            
print("Hard voting clasifier accuracy: ", f1_score(y_test, hvc_predict))

LogisticRegression 0.7747747747747747
RandomForestClassifier 0.6836734693877552
SVC 0.7678571428571429
Hard voting clasifier accuracy:  0.7636363636363637


In [36]:
log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(probability=True, random_state=42)

soft_voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
soft_voting_clf.fit(X, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [39]:
for clf in (log_clf, rnd_clf, svm_clf, soft_voting_clf):
    clf.fit(X, y_train)
    y_pred = clf.predict(X)
#    print(clf.__class__.__name__, f1_score (y_test, y_pred))
    print(clf.__class__.__name__, f1_score (y_train, y_train_pred))

LogisticRegression 0.9647058823529412
RandomForestClassifier 0.9647058823529412
SVC 0.9647058823529412
VotingClassifier 0.9647058823529412


In [100]:
data = 'absolute accordingly nagottu'
data = re.sub('[^a-zA-Z]', ' ', data).lower().split() 
data = [LMT.lemmatize(x) for x in data if not x in set(stopwords.words('english')) - {'not'}]
print(data)
data = Tfidf.transform(data).toarray()
print(data)
print(MB.predict(data))

['absolute', 'accordingly', 'nagottu']
[[1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[1 1 1]


In [88]:
dt = Tfidf.vocabulary_

In [103]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X, y_train)
y_pred = bag_clf.predict(XT)

In [104]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7250996015936255


In [159]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
        DecisionTreeClassifier(max_depth=2), n_estimators=500,
        algorithm="SAMME.R", learning_rate=0.5, random_state=42
    )
ada_clf.fit(X, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                   learning_rate=0.5, n_estimators=500, random_state=42)

In [160]:
from sklearn.metrics import accuracy_score
y_train_pred = ada_clf.predict(X)
accuracy_score(y_train, y_train_pred)

0.988031914893617

In [161]:
from sklearn.metrics import accuracy_score
y_pred = ada_clf.predict(XT)
accuracy_score(y_test, y_pred)

0.7091633466135459